In [1]:
import cv2, time
import torch

from sort import *
from custom_utils import filter_human, draw_rectangle, draw_matching_line, extract_obj_image, calculate_cosine_similarity_matrix, calculate_hungarian_algorithm, calculate_non_assignments, draw_id, Extract_bbox_info, calculate_weighted_similarity_matrix

from re_identification import Global_Id_Identificator
from osnet.feature_extraction import FeatureExtraction
from feature_extractor_torchreid import Feature_Extractor_Torchreid

c:\Users\user\anaconda3\envs\yolov5\lib\site-packages\torchreid\reid\metrics\rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True);
model.conf = 0.3
model.float()
model.eval();

Using cache found in C:\Users\user/.cache\torch\hub\ultralytics_yolov5_master
WARNING  'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING  'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")

YOLOv5  2023-5-24 Python-3.9.18 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [7]:
# feature_extractor = FeatureExtraction(device='cuda')
feature_extractor = Feature_Extractor_Torchreid()

mot_tracker1 = Sort(max_age=500, iou_threshold=0.1, min_hits=1)
mot_tracker2 = Sort(max_age=500, iou_threshold=0.1, min_hits=1)

GII = Global_Id_Identificator()

Model: osnet_ain_x1_0
- params: 2,193,616
- flops: 978,878,352
Successfully loaded pretrained weights from "feature_extractor_weights/osnet_ain_ms_m_c.pth.tar"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [8]:
cam_path_list = ['./test_video/cam1_6.mp4', './test_video/cam2_6.mp4']

vid = [cv2.VideoCapture(i) for i in cam_path_list]

while(True):
    # 이미지 read
    ret1, image_cam1 = vid[0].read() # (h, w, 3)
    ret2, image_cam2 = vid[1].read() 
    
    image_cam1 = cv2.resize(image_cam1, (640, 360)) # (w, h)
    image_cam2 = cv2.resize(image_cam2, (640, 360))
    
    if not ret1 or not ret2: 
        break
    
    # object detection
    preds1 = model(image_cam1);
    preds2 = model(image_cam2);
    dets1 = preds1.pred[0].to('cpu').numpy()
    dets2 = preds2.pred[0].to('cpu').numpy()

    dets1 = filter_human(dets1)
    dets2 = filter_human(dets2)

    draw_image_cam1 = image_cam1.copy()
    draw_image_cam2 = image_cam2.copy()
    
    ## SORT
    trks1 = mot_tracker1.update(dets1)
    trks2 = mot_tracker2.update(dets2)
    
    ## feature extraction
    obj_image_cam1 = extract_obj_image(image_cam1, trks1)
    obj_image_cam2 = extract_obj_image(image_cam2, trks2)
    
    features1, id1 = feature_extractor.prdict_multi_input(obj_image_cam1, trks1[:,-1])
    features2, id2 = feature_extractor.prdict_multi_input(obj_image_cam2, trks2[:,-1])    
    
    ## compare feature using various methods
        ## 1. cosin similarity
    similarity_matrix = calculate_cosine_similarity_matrix(features1, features2)
    weighted_similarity_matrix = calculate_weighted_similarity_matrix(similarity_matrix, trks1, trks2, GII)
    
    assignments, cos_similarities = calculate_hungarian_algorithm(weighted_similarity_matrix)
    matched   = assignments[cos_similarities > 0.55]
    unmatched = calculate_non_assignments(len(features1), len(features2), matched)
    
    ## extract matched/unmatched bbox info
    matched_bbox, unmatched_bbox = Extract_bbox_info(matched, unmatched, trks1, trks2)
    
    ## re-identificationunmatched_bbox_cam2    
    GII.update(matched, unmatched, matched_bbox, unmatched_bbox, (id1, id2))
    GII.prev_id_tables_update()

    
    full_cam = np.concatenate([draw_image_cam1, draw_image_cam2], axis=0)
        
    ## draw rectangle
    draw_rectangle(full_cam, trks1, cam_type=1)
    draw_rectangle(full_cam, trks2, cam_type=2) 
    
    ## draw matching line
    draw_matching_line(full_cam, matched, (trks1, trks2), (id1, id2))
    
    ## draw ID 
    draw_id(full_cam, trks1, trks2, GII)
    
    cv2.imshow('full_cam', full_cam)
    
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break


vid[0].release()
vid[1].release()
cv2.destroyAllWindows()

In [5]:
vid[0].release()
vid[1].release()
cv2.destroyAllWindows()